In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium import webdriver
import time
import pandas as pd
import openpyxl
import re

# web: https://portal.3gpp.org/Specifications.aspx
# date: 2023/4/8

### 1. 下載spec的history資料

In [2]:
df = pd.read_excel("reference/Work_plan_3gpp_230110.xlsm")
SpecificationList_df = pd.read_excel("reference/2023-03-07_1805_SpecificationList_2e6c4e.xlsx")
ws = openpyxl.load_workbook('reference/2023-03-07_1805_SpecificationList_2e6c4e.xlsx')['Specifications']

histories = {}

In [7]:
def get_spec_history(spec_link):
    driver.get(spec_link)
    time.sleep(0.5)
    
    record_elements = driver.find_elements(By.XPATH, "//*[@id='specificationHistory_historyTable_ctl00']//tbody//tr")
    
    edit_dates = []
    pre_rapporteur = []
    new_rapporteur = []

    for i in record_elements:
        if i.get_attribute('class') != 'rgNoRecords':
            try:
                edit_dates.append(i.find_elements(By.XPATH, "td")[0].text.split(' ')[0])
                pre_rapporteur.append(i.find_elements(By.XPATH, "td")[1].text.split(': ')[1].strip())
                new_rapporteur.append(i.find_elements(By.XPATH, "td")[1].text.replace('to', '.').split(".")[1].strip())
            except:
                pass
    
    return {
        'date': edit_dates,
        'previous rapporteur': pre_rapporteur,
        'new rapporteur': new_rapporteur
    }

In [10]:
driver = webdriver.Chrome(ChromeDriverManager().install())

for i in range(1, ws.max_row - 1):
    cur_spec = ws.cell(1 + i, 1)
    if cur_spec.value not in histories.keys():
        histories[cur_spec.value] = get_spec_history(cur_spec.hyperlink.target)

### 2. 整理成dataframe

In [55]:
output_df = pd.DataFrame()

for spec, his in histories.items():
    if his['date'] != []:
        for i in range(len(his['date'])):
            try:
                his_dict = pd.DataFrame({
                    'spec': spec,
                    'data': his['date'][i],
                    'previous rapporteur': his['previous rapporteur'][i],
                    'new rapporteur': his['new rapporteur'][i]
                }, index=[0])
            except:
                pass
            else:
                output_df = pd.concat([output_df, his_dict], axis=0, ignore_index=True)

In [62]:
output_df.replace('(none)', None, regex=True, inplace=True)
output_df.dropna(inplace=True)
output_df.to_excel('history_result.xlsx', encoding='utf_8_sig', index=False)